In [1]:
import numpy as np

In [2]:
def mostrar(path):
    with open(path, 'r') as file:
        imgmap = file.read()
    print(imgmap)
    return imgmap

In [3]:
#Detalles switch: (2)
gr_col = 3
gr_row = 1
col = 6
row = 2

In [4]:
#Abrimos el .map inicial con el rectángulo inicial y los 3 saltos necesarios
path_input = 'imagenes_y_maps/[StorageReview-Mellanox-SX6036-InfiniBand-Switch-Front] (imported).map'
imgmap = mostrar(path_input)

<img src="[StorageReview-Mellanox-SX6036-InfiniBand-Switch-Front] (imported)" width="850" height="129" border="0" usemap="#map" />

<map name="map">
<!-- #$-:Image map file created by GIMP Image Map plug-in -->
<!-- #$-:GIMP Image Map plug-in by Maurits Rijk -->
<!-- #$-:Please do not edit lines starting with "#$" -->
<!-- #$VERSION:2.3 -->
<!-- #$AUTHOR:Gonzalo Contreras Aso -->
<area shape="rect" coords="70,51,105,67" alt="1"  nohref="nohref" />
<area shape="rect" coords="108,51,142,67" alt="3"  nohref="nohref" />
<area shape="rect" coords="71,76,106,92" alt="2"  nohref="nohref" />
<area shape="rect" coords="300,53,334,67" alt="13"  nohref="nohref" />
</map>



In [5]:
#Conseguimos el diccionario con las cordenadas del rectángulo inicial y los saltos necesarios
coords_dict = {}
split = [split.split('\" alt=\"') for split in imgmap.split('coords=\"')[1:]]
for coords in split:
    alt = coords[1].split('\"  nohref')[0] #Careful: dos espacios entre \" y nohref
    coords_dict[int(alt)] = coords[0]
coords_dict

{1: '70,51,105,67', 3: '108,51,142,67', 2: '71,76,106,92', 13: '300,53,334,67'}

In [6]:
dist_dict = {}
pos_base = [int(pos) for pos in coords_dict[1].split(",")]
for entry in coords_dict.keys():
    if entry == 1:
        continue
    pos = [int(pos) for pos in coords_dict[entry].split(",")]
    dist_dict[entry] = np.subtract(pos,pos_base)
dist_dict

{3: array([38,  0, 37,  0]),
 2: array([ 1, 25,  1, 25]),
 13: array([230,   2, 229,   0])}

In [7]:
#Nos vamos a quedar con los dos primeros números de cada: los dos siguientes en un mundo perfecto debería ser iguales, y el bucle se asegurará de ello.
dist_good = {entry: list(dist_dict[entry][:2]) + list(dist_dict[entry][:2]) for entry in dist_dict.keys()}
dist_good

{3: [38, 0, 38, 0], 2: [1, 25, 1, 25], 13: [230, 2, 230, 2]}

In [10]:
alt = 1
all_pos = {}
all_pos[1] = np.array(pos_base)

def calcular_posicion(alt,i,j,k,l):
    #Calculo del siguiente rectángulo.
    change = np.multiply(i,dist_good[2]) + np.multiply(j,dist_good[3]) + np.multiply(k,dist_good[col * row + 1])
    if l: 
        change += np.multiply(l,dist_good[col * row * gr_col + 1]) 
        
    pos = np.add(pos_base, change)
    alt += 1
    return alt, pos 

for l in range(0,gr_row):
    for k in range(0,gr_col):
        for j in range(0,col):
            for i in range(0,row):
                if i == 0 and j == 0 and k == 0 and l == 0:
                    continue
                alt, pos = calcular_posicion(alt,i,j,k,l)
                all_pos[alt] = pos
all_pos

{1: array([ 70,  51, 105,  67]),
 2: array([ 71,  76, 106,  92]),
 3: array([108,  51, 143,  67]),
 4: array([109,  76, 144,  92]),
 5: array([146,  51, 181,  67]),
 6: array([147,  76, 182,  92]),
 7: array([184,  51, 219,  67]),
 8: array([185,  76, 220,  92]),
 9: array([222,  51, 257,  67]),
 10: array([223,  76, 258,  92]),
 11: array([260,  51, 295,  67]),
 12: array([261,  76, 296,  92]),
 13: array([300,  53, 335,  69]),
 14: array([301,  78, 336,  94]),
 15: array([338,  53, 373,  69]),
 16: array([339,  78, 374,  94]),
 17: array([376,  53, 411,  69]),
 18: array([377,  78, 412,  94]),
 19: array([414,  53, 449,  69]),
 20: array([415,  78, 450,  94]),
 21: array([452,  53, 487,  69]),
 22: array([453,  78, 488,  94]),
 23: array([490,  53, 525,  69]),
 24: array([491,  78, 526,  94]),
 25: array([530,  55, 565,  71]),
 26: array([531,  80, 566,  96]),
 27: array([568,  55, 603,  71]),
 28: array([569,  80, 604,  96]),
 29: array([606,  55, 641,  71]),
 30: array([607,  80, 6

In [26]:
#Eliminar los tags que ahora sustituiremos
splitmap = imgmap.split('\n')
finalmap = [] 
for split in splitmap:
    if split.find('<area'):
        finalmap.append(split)
    
finalmap

['<img src="[SX6536_detalle] (imported)" width="538" height="141" border="0" usemap="#map" />',
 '',
 '<map name="map">',
 '<!-- #$-:Image map file created by GIMP Image Map plug-in -->',
 '<!-- #$-:GIMP Image Map plug-in by Maurits Rijk -->',
 '<!-- #$-:Please do not edit lines starting with "#$" -->',
 '<!-- #$VERSION:2.3 -->',
 '<!-- #$AUTHOR:Gonzalo Contreras Aso  -->',
 '</map>',
 '']

In [27]:
# Insertar los tags en la lista que irá al fichero
for entry in all_pos.keys():
    intlist = all_pos[entry].tolist()
    strlist = [str(num) for num in intlist]
    val = ",".join(strlist)
    tag = f'<area shape="rect" coords="{val}" alt="{entry}"  nohref="nohref" />'
    finalmap.insert(-2,tag)
finalmap

['<img src="[SX6536_detalle] (imported)" width="538" height="141" border="0" usemap="#map" />',
 '',
 '<map name="map">',
 '<!-- #$-:Image map file created by GIMP Image Map plug-in -->',
 '<!-- #$-:GIMP Image Map plug-in by Maurits Rijk -->',
 '<!-- #$-:Please do not edit lines starting with "#$" -->',
 '<!-- #$VERSION:2.3 -->',
 '<!-- #$AUTHOR:Gonzalo Contreras Aso  -->',
 '<area shape="rect" coords="108,61,141,76" alt="1"  nohref="nohref" />',
 '<area shape="rect" coords="108,84,141,99" alt="2"  nohref="nohref" />',
 '<area shape="rect" coords="143,61,176,76" alt="3"  nohref="nohref" />',
 '<area shape="rect" coords="143,84,176,99" alt="4"  nohref="nohref" />',
 '<area shape="rect" coords="178,61,211,76" alt="5"  nohref="nohref" />',
 '<area shape="rect" coords="178,84,211,99" alt="6"  nohref="nohref" />',
 '<area shape="rect" coords="219,61,252,76" alt="7"  nohref="nohref" />',
 '<area shape="rect" coords="219,84,252,99" alt="8"  nohref="nohref" />',
 '<area shape="rect" coords="25

In [28]:
outfile = 'imagenes_y_maps/SX6536_detalle.map'
with open(outfile, 'w') as file:
    file.write('\n'.join(finalmap))